<a href="https://colab.research.google.com/github/jiang28/Deep-Spatio-Temporal-Encoding/blob/master/Heavy_rain_risk_TransGcnRE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/

/content/drive/MyDrive


In [ ]:
PATH = '/content/drive/My Drive/'
device = 'cuda'
import torch
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'
print(device)

cuda


In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from pandas import read_csv
import pandas as pd

output_window = 1

def get_data():
    data = []
    features = []
    location = []
    label = []

    # Load npy files
    df = read_csv('Rain/data_total_prep2.csv', sep=',', header=None, low_memory=False, keep_default_na=False)

    # Convert the DataFrame to a NumPy array
    series = df.to_numpy()

    for i in range(len(df)):
        if i == 0:
            continue

        line = series[i][1:-14]
        line = [float(_) if _ != '' else -1 for _ in line]
        line = [int(j) for j in line]

        data.append(line)

        f = series[i][-14:-1]
        f = [float(_) if _ != '' else -1 for _ in f]
        f = [int(_) for _ in f]

        features.append(f)

        label.append(int(float(series[i][-1])))

    data = np.asarray(data)
    features = np.asarray(features)
    label = np.asarray(label)

    data_ = []
    features_ = []
    for i in range(len(data)):
        scaler = MinMaxScaler(feature_range=(0, 1))
        x = data[i]
        x = scaler.fit_transform(np.array(x).reshape(-1, 1)).reshape(-1)
        data_.append(x)

        features_.append(scaler.fit_transform(np.array(features[i]).reshape(-1, 1)).reshape(-1))

    input_data = torch.FloatTensor(data_)
    label = torch.FloatTensor(label)
    features = torch.FloatTensor(features_)

    # Shuffle the data before splitting into train and test sets
    x_train, x_test, y_train, y_test, ftr_train, ftr_test = train_test_split(
        input_data, label, features, test_size=0.3, random_state=42, shuffle=True)

    return x_train, y_train, ftr_train, x_test, y_test, ftr_test, features

if __name__ == '__main__':
    x_train, y_train, ftr_train, x_test, y_test, ftr_test, ftr_all = get_data()
    print(x_train.shape)
    print(x_train)
    print(y_train.shape)
    print(ftr_train.shape)

    print(x_test.shape)
    print(y_test.shape)
    print(ftr_test.shape)

    print(ftr_all.shape)

    pass


torch.Size([7000, 132])
tensor([[0.0000, 0.0000, 0.0000,  ..., 0.5036, 0.2628, 0.2555],
        [0.0000, 0.0000, 0.0000,  ..., 1.0000, 0.4706, 0.3235],
        [0.0000, 0.0000, 0.0000,  ..., 0.7534, 1.0000, 0.6027],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.5574, 0.3770, 0.2869],
        [0.0000, 0.0000, 0.0000,  ..., 0.0139, 0.0278, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.9583, 0.5104, 0.4896]])
torch.Size([7000])
torch.Size([7000, 13])
torch.Size([3000, 132])
torch.Size([3000])
torch.Size([3000, 13])
torch.Size([10000, 13])


<ipython-input-4-4050b42d2562>:54: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  input_data = torch.FloatTensor(data_)


In [ ]:
#load data
import torch
def load_matrix():
    matrix = []
    with open('Rain/adj_matrix.txt', 'r') as f:
        for i, line in enumerate(f):
            row = [int(x) for x in line.split()]
            matrix.append(row)

    adj = torch.FloatTensor(matrix)
    print('adj_matrix:', adj)
    print('adj_matrix shape:', adj.shape)

    return adj

if __name__ == '__main__':
    adj = load_matrix()
    print('Overall adj shape:', adj.shape)

adj_matrix: tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.]])
adj_matrix shape: torch.Size([10000, 10000])
Overall adj shape: torch.Size([10000, 10000])


In [ ]:
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import argparse
import sys
import math
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super(GCNLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        pe.requires_grad = True
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]

In [ ]:
class TransGCN(nn.Module):
    def __init__(self, input_size, feature_size=250,num_layers=1,dropout=0.1):
        super(TransGCN, self).__init__()

        #self.gc1 = GCNLayer(10, 10)
        self.gc1 = GCNLayer(13, 13)
        # self.gc2 = GCNLayer(5, 250)

        self.src_mask = None
        self.pos_encoder = PositionalEncoding(feature_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=10, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

        self.decoder = nn.Linear(feature_size,1)
        self.layer1 = nn.Linear(input_size, 200)
        self.layer2 = nn.Linear(200, 100)
        self.layer3 = nn.Linear(100, 1)

        self.last_layer = nn.Sigmoid()

    def forward(self, src, ftr_all, adj, st_idx, ed_idx):
        #gcn embedding loc information
        #print('gcn input shape', ftr_all.shape)
        gcn_output = self.gc1(ftr_all, adj)
        #print('gcn output shape', gcn_output.shape)
        gcn_output = F.relu(gcn_output)
        # gcn_tmp = F.relu(self.gc1(ftr_all, adj))
        # gcn_output = self.gc2(gcn_tmp, adj)

        #transformer embedding time series data
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        # junwei
        #print('before pos', src.shape)
        src = self.pos_encoder(src)
        # junwei
        #print('after pos', src.shape)

        hidden = self.transformer_encoder(src,self.src_mask)

        # junwei
        #print('after encoder', src.shape)

        gcn_output = gcn_output[st:ed]
        gcn_output = torch.transpose(gcn_output, 0, 1)

        gcn_output = torch.reshape(gcn_output, (gcn_output.shape[0], gcn_output.shape[1], 1))
        gcn_output = gcn_output.expand(gcn_output.shape[0], gcn_output.shape[1], hidden.shape[2])

        # concate gcn output to transformer output
        output = torch.cat((hidden, gcn_output), 0)

        output = self.decoder(output)

        output = torch.squeeze(output)
        output = output.transpose(0, 1)

        #MLP prediction
        output = self.layer1(output)
        output = self.layer2(output)
        output = self.layer3(output)

        output = self.last_layer(output)
        #print('output', output)
        return output

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [ ]:
if __name__ == '__main__':
    arg_parser = argparse.ArgumentParser()
    arg_parser.add_argument('--mode', choices=['train', 'infer'],\
      default='train',help='Run mode')
    arg_parser.add_argument('--epoch', default='200', type=int)
    arg_parser.add_argument('--batch_size', default='32', type=int)
    args = arg_parser.parse_args(args=['--mode', 'train'])
    args = arg_parser.parse_args(args=['--epoch', '200'])
    #args = arg_parser.parse_args(args=['--epoch', '30'])
    args = arg_parser.parse_args(args=['--batch_size', '32'])

    X_train, y_train, FTR_train, X_test, y_test, FTR_test, FTR_all = get_data()

    X_train = X_train.to(device)
    y_train = y_train.to(device)
    FTR_train = FTR_train.to(device)
    X_test = X_test.to(device)
    y_test = y_test.to(device)
    FTR_test = FTR_test.to(device)

    ftr_all = FTR_all.to(device)

    adj = load_matrix().to(device)

    model_path = PATH + 'MODEL1/model_trans_gcn.pt'
    model = TransGCN(input_size=X_train.shape[1] + ftr_all.shape[1]).to(device)

    loss_function = nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.02)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.98)

    num_batch = int(len(y_train) / args.batch_size) + 1
    num_batch_test = int(len(y_test) / args.batch_size) + 1

    last_acc = 0.0

    if args.mode == 'train':
        for epoch in range(args.epoch):
            model.train()
            acc = []
            total_loss = 0
            acc_test = []

            for i in range(num_batch):
                sys.stdout.write('\r{0}/{1}'.format(i, num_batch))
                st = i * args.batch_size
                ed = min((i + 1) * args.batch_size, len(y_train))

                x = X_train[st:ed].transpose(0, 1).unsqueeze(-1)
                label = y_train[st:ed]

                optimizer.zero_grad()
                output = model(x, ftr_all, adj, st, ed)
                predict = torch.squeeze(output)

                #loss = loss_function(predict+0.00001, label)
                loss = loss_function(predict, label)

                acc.append(torch.sum(predict.gt(0.5) == label))

                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            with torch.no_grad():
                model.eval()
                for i in range(num_batch_test):
                    st = i * args.batch_size
                    ed = min((i + 1) * args.batch_size, len(y_test))

                    x_test = X_test[st:ed].transpose(0, 1).unsqueeze(-1)
                    label_test = y_test[st:ed]

                    output_test = model(x_test, ftr_all, adj, X_train.shape[0] + st, X_train.shape[0] + ed)
                    predict_test = torch.squeeze(output_test)
                    acc_test.append(torch.sum(predict_test.gt(0.5) == label_test))

            total_loss /= len(y_train)
            acc = sum(acc) * 1.0 / len(y_train)
            acc_test = sum(acc_test) * 1.0 / len(y_test)

            if epoch % 10 == 0 or epoch == args.epoch - 1:
                print('\nEpoch: ', epoch)
                print('\nTraining set: Loss {0:.4f}. Acc {1:.4f}.\nTest set: Acc {2:.4f}.'.format(total_loss, acc, acc_test))

        scheduler.step()

        if acc_test > last_acc:
            torch.save(model.cpu(), model_path)
            last_acc = acc_test


adj_matrix: tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.]])
adj_matrix shape: torch.Size([10000, 10000])
218/219
Epoch:  0

Training set: Loss 0.0217. Acc 0.5204.
Test set: Acc 0.5073.
218/219
Epoch:  10

Training set: Loss 0.0147. Acc 0.7606.
Test set: Acc 0.7553.
218/219
Epoch:  20

Training set: Loss 0.0132. Acc 0.7916.
Test set: Acc 0.7833.
218/219
Epoch:  30

Training set: Loss 0.0125. Acc 0.8066.
Test set: Acc 0.7990.
218/219
Epoch:  40

Training set: Loss 0.0120. Acc 0.8194.
Test set: Acc 0.8237.
218/219
Epoch:  50

Training set: Loss 0.0116. Acc 0.8287.
Test set: Acc 0.8283.
218/219
Epoch:  60

Training set: Loss 0.0114. Acc 0.8359.
Test set: Acc 0.8303.
218/219
Epoch:  70

Training set: Loss 0.0110. Acc 0.8397.
Test set: Acc 0.8367.
218/219
Epoch:  80

Training set: Loss 0.0

In [ ]:
if __name__ == '__main__':
    X_train, y_train, FTR_train, X_test, y_test, FTR_test, FTR_all = get_data()

    num_batch = int(len(y_train)/args.batch_size) + 1

    num_batch_test = int(len(y_test)/args.batch_size) + 1

In [ ]:
#TransGCN evaluation
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

model_path = PATH + 'MODEL1/model_trans_gcn.pt'
model = torch.load(model_path)
model.to(device)
acc_test = []
y_pred = []

for i in range(num_batch_test):
    st = i * args.batch_size
    ed = min((i+1) * args.batch_size, len(y_test))

    x_test = X_test[st:ed]
    x_test = x_test.transpose(0, 1)
    x_test = torch.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
    x_test = x_test.to(device)

    label_test = y_test[st:ed]
    label_test = label_test.to(device)

    output_test = model(x_test, ftr_all, adj, X_train.shape[0] + st, X_train.shape[0] + ed)
    predict_test = torch.squeeze(output_test)
    #binary
    y_pred += predict_test.gt(0.5)

    #actual
    #y_pred += predict_test

y_pred = list(map(float, y_pred))
y_pred = np.asarray(y_pred)
y_pred = torch.FloatTensor(y_pred)

# Convert y_pred to a NumPy array
y_pred_np = np.array(y_pred)

# Convert y_test to a NumPy array
y_test_np = np.array(y_test)

# Count the number of 1s and 0s in y_pred and y_test
num_ones_pred = np.count_nonzero(y_pred_np == 1)
num_zeros_pred = np.count_nonzero(y_pred_np == 0)

num_ones_test = np.count_nonzero(y_test_np == 1)
num_zeros_test = np.count_nonzero(y_test_np == 0)

# Print the results
print("Number of 1s in y_pred:", num_ones_pred)
print("Number of 0s in y_pred:", num_zeros_pred)

print("Number of 1s in y_test:", num_ones_test)
print("Number of 0s in y_test:", num_zeros_test)

print('Transformer Results:')
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print('Roc:', metrics.roc_auc_score(y_test, y_pred))

Number of 1s in y_pred: 1615
Number of 0s in y_pred: 1385
Number of 1s in y_test: 1478
Number of 0s in y_test: 1522
Transformer Results:
Accuracy: 0.8663333333333333
[[1253  269]
 [ 132 1346]]
              precision    recall  f1-score   support

         0.0       0.90      0.82      0.86      1522
         1.0       0.83      0.91      0.87      1478

    accuracy                           0.87      3000
   macro avg       0.87      0.87      0.87      3000
weighted avg       0.87      0.87      0.87      3000

Roc: 0.8669744958471066


In [ ]:
#load data
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from pandas import read_csv

output_window = 1

def get_data():
    data = []
    features = []
    location = []
    label = []
    series = read_csv('Rain/data_total_prep2.csv', sep=',', header=None, low_memory=False, keep_default_na=False).to_numpy()

    for i in range(len(series)):
        if i == 0:
            continue

        # if '' in series[i]:
        #   continue

        line = series[i][2:-14]
        line = [float(_) if _ != '' else -1 for _ in line]
        line = [int(j) for j in line]

        data.append(line)

        f = series[i][-14:-1]
        f = [float(_) if _ != '' else -1 for _ in f]
        f = [int(_) for _ in f]

        features.append(f)

        label.append(int(float(series[i][-1])))

    data = np.asarray(data)
    features = np.asarray(features)
    label = np.asarray(label)

    data_ = []
    features_ = []
    for i in range(len(data)):
        scaler = MinMaxScaler(feature_range=(0, 1))
        x = data[i]
        x = scaler.fit_transform(np.array(x).reshape(-1, 1)).reshape(-1)
        data_.append(x)

        features_.append(scaler.fit_transform(np.array(features[i]).reshape(-1, 1)).reshape(-1))

    input_data = torch.FloatTensor(data_)
    label = torch.FloatTensor(label)
    features = torch.FloatTensor(features_)

    samples = int(len(input_data)*0.7)
    x_train = input_data[:samples]
    x_test = input_data[samples:]

    y_train = label[:samples]
    y_test = label[samples:]

    ftr_train = features[:samples]
    ftr_test = features[samples:]

    return x_train, y_train, ftr_train, x_test, y_test, ftr_test

if __name__ == '__main__':
    x_train, y_train, ftr_train, x_test, y_test, ftr_test = get_data()
    print(x_train.shape)
    print(y_train.shape)
    print(ftr_train.shape)

    print(x_test.shape)
    print(y_test.shape)
    print(ftr_test.shape)

    pass


torch.Size([7000, 131])
torch.Size([7000])
torch.Size([7000, 13])
torch.Size([3000, 131])
torch.Size([3000])
torch.Size([3000, 13])


In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from pandas import read_csv
import pandas as pd

output_window = 1

def get_data():
    data = []
    features = []
    location = []
    label = []

    # Load npy files
    df = read_csv('Rain/data_total_prep2.csv', sep=',', header=None, low_memory=False, keep_default_na=False)

    # Convert the DataFrame to a NumPy array
    series = df.to_numpy()

    for i in range(len(df)):
        if i == 0:
            continue

        line = series[i][1:-14]
        line = [float(_) if _ != '' else -1 for _ in line]
        line = [int(j) for j in line]

        data.append(line)

        f = series[i][-14:-1]
        f = [float(_) if _ != '' else -1 for _ in f]
        f = [int(_) for _ in f]

        features.append(f)

        label.append(int(float(series[i][-1])))

    data = np.asarray(data)
    features = np.asarray(features)
    label = np.asarray(label)

    data_ = []
    features_ = []
    for i in range(len(data)):
        scaler = MinMaxScaler(feature_range=(0, 1))
        x = data[i]
        x = scaler.fit_transform(np.array(x).reshape(-1, 1)).reshape(-1)
        data_.append(x)

        features_.append(scaler.fit_transform(np.array(features[i]).reshape(-1, 1)).reshape(-1))

    input_data = torch.FloatTensor(data_)
    label = torch.FloatTensor(label)
    features = torch.FloatTensor(features_)

    # Shuffle the data before splitting into train and test sets
    x_train, x_test, y_train, y_test, ftr_train, ftr_test = train_test_split(
        input_data, label, features, test_size=0.3, random_state=42, shuffle=True)

    return x_train, y_train, ftr_train, x_test, y_test, ftr_test

if __name__ == '__main__':
    x_train, y_train, ftr_train, x_test, y_test, ftr_test = get_data()
    print(x_train.shape)
    print(x_train)
    print(y_train.shape)
    print(ftr_train.shape)

    print(x_test.shape)
    print(y_test.shape)
    print(ftr_test.shape)

    print(ftr_all.shape)

    pass


torch.Size([7000, 132])
tensor([[0.0000, 0.0000, 0.0000,  ..., 0.5036, 0.2628, 0.2555],
        [0.0000, 0.0000, 0.0000,  ..., 1.0000, 0.4706, 0.3235],
        [0.0000, 0.0000, 0.0000,  ..., 0.7534, 1.0000, 0.6027],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.5574, 0.3770, 0.2869],
        [0.0000, 0.0000, 0.0000,  ..., 0.0139, 0.0278, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.9583, 0.5104, 0.4896]])
torch.Size([7000])
torch.Size([7000, 13])
torch.Size([3000, 132])
torch.Size([3000])
torch.Size([3000, 13])
torch.Size([10000, 13])


In [ ]:
##Transformer
from __future__ import print_function
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import argparse
import sys
import math

class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=5000,**block_args):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        pe.requires_grad = True
        self.register_buffer('pe', pe)
        #self.layers = nn.ModuleList([EncoderBlock(**block_args) for _ in range(num_layers)])

    def forward(self, x):
        return x + self.pe[:x.size(0), :]

    def get_attention_maps(self, x, mask=None):
        attention_maps = []
        for l in self.layers:
            _, attn_map = l.self_attn(x, mask=mask, return_attention=True)
            attention_maps.append(attn_map)
            x = l(x)
        return attention_maps

class Transformer(nn.Module):
    def __init__(self, input_size, feature_size=250,num_layers=1,dropout=0.1):
        super(Transformer, self).__init__()
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(feature_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=10, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

        self.layer = nn.Linear(feature_size,1)
        self.layer1 = nn.Linear(input_size, 200)
        self.layer2 = nn.Linear(200, 100)
        self.layer3 = nn.Linear(100, 1)

        self.last_layer = nn.Sigmoid()


    def forward(self, src, ftr):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.pos_encoder(src)
        hidden = self.transformer_encoder(src,self.src_mask)

        ftr = torch.reshape(ftr, (ftr.shape[0], ftr.shape[1], 1))
        ftr = ftr.expand(ftr.shape[0], ftr.shape[1], hidden.shape[2])

        hidden = torch.cat((hidden, ftr), 0)

        output = self.layer(hidden)

        output = torch.squeeze(output)
        output = output.transpose(0, 1)

        output = self.layer1(output)
        output = self.layer2(output)
        output = self.layer3(output)

        output = self.last_layer(output)
        return output

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask


if __name__ == '__main__':
    arg_parser = argparse.ArgumentParser()
    arg_parser.add_argument('--mode', choices=['train', 'infer'],\
        default='train',help='Run mode')
    arg_parser.add_argument('--epoch', default='200', type=int)
    arg_parser.add_argument('--batch_size', default='32', type=int)
    args = arg_parser.parse_args(args=['--mode', 'train'])
    args = arg_parser.parse_args(args=['--epoch', '200'])
    args = arg_parser.parse_args(args=['--batch_size', '32'])

    model_path = PATH + 'MODEL1/model_all_transformer_12.pt'

    X_train, y_train, FTR_train, X_test, y_test, FTR_test = get_data()

    print(X_train.shape)
    print(y_train.shape)
    print(X_test.shape)
    print(y_test.shape)

    model = Transformer(input_size = X_train.shape[1] + FTR_train.shape[1])
    #loss_function = nn.MSELoss()
    loss_function = nn.BCELoss()

    #map = model.get_attention_maps(X_train)
    #print(map)

    optimizer = optim.SGD(model.parameters(), lr=0.02)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.98)

    num_batch = int(len(y_train)/args.batch_size) + 1

    num_batch_test = int(len(y_test)/args.batch_size) + 1

    last_acc = 0.0

    if args.mode == 'train':

        for epoch in range(args.epoch):

            acc = []
            total_loss = 0

            acc_test = []

            model.to(device)

            for i in range(num_batch):
                sys.stdout.write('\r{0}/{1}'.format(i, num_batch))
                st = i * args.batch_size
                ed = min((i+1) * args.batch_size, len(y_train))

                x = X_train[st:ed]
                x = x.transpose(0, 1)
                x = torch.reshape(x, (x.shape[0], x.shape[1], 1))
                x = x.to(device)

                ftr = FTR_train[st:ed]
                ftr = ftr.transpose(0, 1)
                ftr = torch.reshape(ftr, (ftr.shape[0], ftr.shape[1], 1))
                ftr = ftr.to(device)

                label = y_train[st:ed]
                label = label.to(device)

                model.zero_grad()

                output = model(x, ftr)
                predict = torch.squeeze(output)

                loss = loss_function(predict, label)

                acc.append(torch.sum(predict.gt(0.5) == label))
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            for i in range(num_batch_test):
                st = i * args.batch_size
                ed = min((i+1) * args.batch_size, len(y_test))

                x_test = X_test[st:ed]
                x_test = x_test.transpose(0, 1)
                x_test = torch.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
                x_test = x_test.to(device)

                ftr_test = FTR_test[st:ed]
                ftr_test = ftr_test.transpose(0, 1)
                ftr_test = torch.reshape(ftr_test, (ftr_test.shape[0], ftr_test.shape[1], 1))
                ftr_test = ftr_test.to(device)

                label_test = y_test[st:ed]
                label_test = label_test.to(device)


                output_test = model(x_test, ftr_test)
                predict_test = torch.squeeze(output_test)
                acc_test.append(torch.sum(predict_test.gt(0.5) == label_test))

            total_loss /= len(y_train)
            acc = sum(acc)*1.0/len(y_train)

            acc_test = sum(acc_test)*1.0/len(y_test)
            if epoch%10 == 0 or epoch == args.epoch - 1:
              print('\nEpoch: ', epoch)
              print('\nTraining set: Loss {0:.4f}. Acc {1:.4f}.\nTest set: Acc {2:.4f}.'.format(total_loss, acc, acc_test))

            scheduler.step()

            if acc_test > last_acc:
                torch.save(model.cpu(), model_path)
                last_acc = acc_test

torch.Size([7000, 132])
torch.Size([7000])
torch.Size([3000, 132])
torch.Size([3000])
218/219
Epoch:  0

Training set: Loss 0.0217. Acc 0.5251.
Test set: Acc 0.5073.
218/219
Epoch:  10

Training set: Loss 0.0150. Acc 0.7537.
Test set: Acc 0.7297.
218/219
Epoch:  20

Training set: Loss 0.0136. Acc 0.7834.
Test set: Acc 0.7750.
218/219
Epoch:  30

Training set: Loss 0.0128. Acc 0.7999.
Test set: Acc 0.7910.
218/219
Epoch:  40

Training set: Loss 0.0122. Acc 0.8127.
Test set: Acc 0.8040.
218/219
Epoch:  50

Training set: Loss 0.0118. Acc 0.8183.
Test set: Acc 0.8133.
218/219
Epoch:  60

Training set: Loss 0.0115. Acc 0.8276.
Test set: Acc 0.8220.
218/219
Epoch:  70

Training set: Loss 0.0112. Acc 0.8354.
Test set: Acc 0.8223.
218/219
Epoch:  80

Training set: Loss 0.0111. Acc 0.8387.
Test set: Acc 0.8283.
218/219
Epoch:  90

Training set: Loss 0.0109. Acc 0.8373.
Test set: Acc 0.8250.
218/219
Epoch:  100

Training set: Loss 0.0108. Acc 0.8441.
Test set: Acc 0.8330.
218/219
Epoch:  110

Tr

In [ ]:
#Transformer evaluation
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

model_path = PATH + 'MODEL1/model_all_transformer_12.pt'
model = torch.load(model_path)
model.to(device)
acc_test = []
y_pred = []

for i in range(num_batch_test):
    st = i * args.batch_size
    ed = min((i+1) * args.batch_size, len(y_test))

    x_test = X_test[st:ed]
    x_test = x_test.transpose(0, 1)
    x_test = torch.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
    x_test = x_test.to(device)

    ftr_test = FTR_test[st:ed]
    ftr_test = ftr_test.transpose(0, 1)
    ftr_test = torch.reshape(ftr_test, (ftr_test.shape[0], ftr_test.shape[1], 1))
    ftr_test = ftr_test.to(device)

    label_test = y_test[st:ed]
    label_test = label_test.to(device)


    output_test = model(x_test, ftr_test)
    predict_test = torch.squeeze(output_test)
    y_pred += predict_test.gt(0.5)

y_pred = list(map(float, y_pred))
y_pred = np.asarray(y_pred)
y_pred = torch.FloatTensor(y_pred)

print('Transformer Results:')
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(metrics.roc_auc_score(y_test, y_pred))
# calculate the fpr and tpr for all thresholds of the classification

#fpr, tpr, threshold = metrics.roc_curve(y_test, y_pred)
#roc_auc = metrics.auc(fpr, tpr)

# method I: plt
#import matplotlib.pyplot as plt

#plt.title('Receiver Operating Characteristic')
#plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
#plt.legend(loc = 'lower right')
#plt.plot([0, 1], [0, 1],'r--',label='Sample Label Red')
#plt.xlim([0, 1])
#plt.ylim([0, 1])
#plt.ylabel('True Positive Rate')
#plt.xlabel('False Positive Rate')
#plt.savefig(PATH+"roc.png")


Transformer Results:
Accuracy: 0.8346666666666667
[[1249  273]
 [ 223 1255]]
              precision    recall  f1-score   support

         0.0       0.85      0.82      0.83      1522
         1.0       0.82      0.85      0.83      1478

    accuracy                           0.83      3000
   macro avg       0.83      0.83      0.83      3000
weighted avg       0.84      0.83      0.83      3000

0.834875591016023


In [ ]:
#MLP baseline
from __future__ import print_function
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import argparse
import sys
import math

class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.layer1 = nn.Linear(input_size, 200)
        self.layer2 = nn.Linear(200, 100)
        self.layer3 = nn.Linear(100, 1)

        self.last_layer = nn.Sigmoid()

    def forward(self, src, ftr):
        ftr = torch.reshape(ftr, (ftr.shape[0], ftr.shape[1], 1))
        ftr = ftr.expand(ftr.shape[0], ftr.shape[1], src.shape[2])

        hidden = torch.cat((src, ftr), 0)

        output = torch.squeeze(hidden)
        output = output.transpose(0, 1)

        output = self.layer1(output)
        output = self.layer2(output)
        output = self.layer3(output)

        output = self.last_layer(output)
        return output

if __name__ == '__main__':
    arg_parser = argparse.ArgumentParser()
    arg_parser.add_argument('--mode', choices=['train', 'infer'],\
        default='train',help='Run mode')
    arg_parser.add_argument('--epoch', default='60', type=int)
    arg_parser.add_argument('--batch_size', default='32', type=int)
    args = arg_parser.parse_args(args=['--mode', 'train'])
    args = arg_parser.parse_args(args=['--epoch', '60'])
    args = arg_parser.parse_args(args=['--batch_size', '32'])

    model_path_mlp = PATH + 'MODEL1/model_mlp_temporal.pt'

    X_train, y_train, FTR_train, X_test, y_test, FTR_test = get_data()

    print(X_train.shape)
    print(y_train.shape)
    print(X_test.shape)
    print(y_test.shape)

    model = MLP(input_size = X_train.shape[1] + FTR_train.shape[1])
    #loss_function = nn.MSELoss()
    loss_function = nn.BCELoss()

    optimizer = optim.SGD(model.parameters(), lr=0.01)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.98)

    num_batch = int(len(y_train)/args.batch_size) + 1

    num_batch_test = int(len(y_test)/args.batch_size) + 1

    last_acc = 0.0

    if args.mode == 'train':

        for epoch in range(args.epoch):

            acc = []
            total_loss = 0

            acc_test = []

            model.to(device)

            for i in range(num_batch):
                sys.stdout.write('\r{0}/{1}'.format(i, num_batch))
                st = i * args.batch_size
                ed = min((i+1) * args.batch_size, len(y_train))

                x = X_train[st:ed]
                x = x.transpose(0, 1)
                x = torch.reshape(x, (x.shape[0], x.shape[1], 1))
                x = x.to(device)

                ftr = FTR_train[st:ed]
                ftr = ftr.transpose(0, 1)
                ftr = torch.reshape(ftr, (ftr.shape[0], ftr.shape[1], 1))
                ftr = ftr.to(device)

                label = y_train[st:ed]
                label = label.to(device)

                model.zero_grad()

                output = model(x, ftr)
                predict = torch.squeeze(output)

                loss = loss_function(predict, label)

                acc.append(torch.sum(predict.gt(0.5) == label))
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            for i in range(num_batch_test):
                st = i * args.batch_size
                ed = min((i+1) * args.batch_size, len(y_test))

                x_test = X_test[st:ed]
                x_test = x_test.transpose(0, 1)
                x_test = torch.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
                x_test = x_test.to(device)

                ftr_test = FTR_test[st:ed]
                ftr_test = ftr_test.transpose(0, 1)
                ftr_test = torch.reshape(ftr_test, (ftr_test.shape[0], ftr_test.shape[1], 1))
                ftr_test = ftr_test.to(device)

                label_test = y_test[st:ed]
                label_test = label_test.to(device)


                output_test = model(x_test, ftr_test)
                predict_test = torch.squeeze(output_test)
                acc_test.append(torch.sum(predict_test.gt(0.5) == label_test))

            total_loss /= len(y_train)
            acc = sum(acc)*1.0/len(y_train)

            acc_test = sum(acc_test)*1.0/len(y_test)
            if epoch%10 == 0 or epoch == args.epoch - 1:
              print('\nEpoch: ', epoch)
              print('\nTraining set: Loss {0:.4f}. Acc {1:.4f}.\nTest set: Acc {2:.4f}.'.format(total_loss, acc, acc_test))

            scheduler.step()

            if acc_test > last_acc:
                torch.save(model.cpu(), model_path_mlp)
                last_acc = acc_test

torch.Size([7000, 132])
torch.Size([7000])
torch.Size([3000, 132])
torch.Size([3000])
218/219
Epoch:  0

Training set: Loss 0.0216. Acc 0.5657.
Test set: Acc 0.5167.
218/219
Epoch:  10

Training set: Loss 0.0166. Acc 0.7480.
Test set: Acc 0.7497.
218/219
Epoch:  20

Training set: Loss 0.0142. Acc 0.7583.
Test set: Acc 0.7553.
218/219
Epoch:  30

Training set: Loss 0.0134. Acc 0.7853.
Test set: Acc 0.7803.
218/219
Epoch:  40

Training set: Loss 0.0129. Acc 0.8001.
Test set: Acc 0.7943.
218/219
Epoch:  50

Training set: Loss 0.0127. Acc 0.8044.
Test set: Acc 0.7990.
218/219
Epoch:  59

Training set: Loss 0.0125. Acc 0.8081.
Test set: Acc 0.8017.


In [ ]:
#MLP evaluation
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

model = torch.load(model_path_mlp)
model.to(device)
acc_test = []
y_pred = []

for i in range(num_batch_test):
    st = i * args.batch_size
    ed = min((i+1) * args.batch_size, len(y_test))

    x_test = X_test[st:ed]
    x_test = x_test.transpose(0, 1)
    x_test = torch.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
    x_test = x_test.to(device)

    ftr_test = FTR_test[st:ed]
    ftr_test = ftr_test.transpose(0, 1)
    ftr_test = torch.reshape(ftr_test, (ftr_test.shape[0], ftr_test.shape[1], 1))
    ftr_test = ftr_test.to(device)

    label_test = y_test[st:ed]
    label_test = label_test.to(device)


    output_test = model(x_test, ftr_test)
    predict_test = torch.squeeze(output_test)
    y_pred += predict_test.gt(0.5)

y_pred = list(map(float, y_pred))
y_pred = np.asarray(y_pred)
y_pred = torch.FloatTensor(y_pred)

print('MLP Results:')
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(metrics.roc_auc_score(y_test, y_pred))

MLP Results:
Accuracy: 0.802
[[1168  354]
 [ 240 1238]]
              precision    recall  f1-score   support

         0.0       0.83      0.77      0.80      1522
         1.0       0.78      0.84      0.81      1478

    accuracy                           0.80      3000
   macro avg       0.80      0.80      0.80      3000
weighted avg       0.80      0.80      0.80      3000

0.8025148520837371


In [ ]:
#LSTM baseline
from __future__ import print_function
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import argparse
import sys
import math

class LSTM(nn.Module):
    def __init__(self, input_size, feature_size=250,num_layers=1,dropout=0.1):
        super(LSTM, self).__init__()
        self.layer1 = nn.LSTM(input_size, 50)
        self.layer2 = nn.Linear(50, 1)

        self.last_layer = nn.Sigmoid()

    def forward(self, src, ftr):

        ftr = torch.reshape(ftr, (ftr.shape[0], ftr.shape[1], 1))
        ftr = ftr.expand(ftr.shape[0], ftr.shape[1], src.shape[2])

        hidden = torch.cat((src, ftr), 0)
        hidden = hidden.transpose(1, 2)
        hidden = hidden.transpose(0, 2)

        hidden, (hn, cn) = self.layer1(hidden)
        output = self.layer2(hidden)

        output = self.last_layer(output)
        return output

if __name__ == '__main__':
    arg_parser = argparse.ArgumentParser()
    arg_parser.add_argument('--mode', choices=['train', 'infer'],\
        default='train',help='Run mode')
    arg_parser.add_argument('--epoch', default='40', type=int)
    arg_parser.add_argument('--batch_size', default='32', type=int)
    args = arg_parser.parse_args(args=['--mode', 'train'])
    args = arg_parser.parse_args(args=['--epoch', '40'])
    args = arg_parser.parse_args(args=['--batch_size', '32'])

    model_path_lstm = PATH + 'MODEL1/model_lstm.pt'

    X_train, y_train, FTR_train, X_test, y_test, FTR_test = get_data()

    print(X_train.shape)
    print(y_train.shape)
    print(X_test.shape)
    print(y_test.shape)

    model = LSTM(input_size = X_train.shape[1] + FTR_train.shape[1])
    #loss_function = nn.MSELoss()
    loss_function = nn.BCELoss()

    optimizer = optim.SGD(model.parameters(), lr=0.1)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.98)

    num_batch = int(len(y_train)/args.batch_size) + 1

    num_batch_test = int(len(y_test)/args.batch_size) + 1

    last_acc = 0.0

    if args.mode == 'train':

        for epoch in range(args.epoch):

            acc = []
            total_loss = 0

            acc_test = []

            model.to(device)

            for i in range(num_batch):
                sys.stdout.write('\r{0}/{1}'.format(i, num_batch))
                st = i * args.batch_size
                ed = min((i+1) * args.batch_size, len(y_train))

                x = X_train[st:ed]
                x = x.transpose(0, 1)
                x = torch.reshape(x, (x.shape[0], x.shape[1], 1))
                x = x.to(device)

                ftr = FTR_train[st:ed]
                ftr = ftr.transpose(0, 1)
                ftr = torch.reshape(ftr, (ftr.shape[0], ftr.shape[1], 1))
                ftr = ftr.to(device)

                label = y_train[st:ed]
                label = label.to(device)

                model.zero_grad()

                output = model(x, ftr)
                predict = torch.squeeze(output)

                loss = loss_function(predict, label)

                acc.append(torch.sum(predict.gt(0.5) == label))
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            for i in range(num_batch_test):
                st = i * args.batch_size
                ed = min((i+1) * args.batch_size, len(y_test))

                x_test = X_test[st:ed]
                x_test = x_test.transpose(0, 1)
                x_test = torch.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
                x_test = x_test.to(device)

                ftr_test = FTR_test[st:ed]
                ftr_test = ftr_test.transpose(0, 1)
                ftr_test = torch.reshape(ftr_test, (ftr_test.shape[0], ftr_test.shape[1], 1))
                ftr_test = ftr_test.to(device)

                label_test = y_test[st:ed]
                label_test = label_test.to(device)


                output_test = model(x_test, ftr_test)
                predict_test = torch.squeeze(output_test)
                acc_test.append(torch.sum(predict_test.gt(0.5) == label_test))

            total_loss /= len(y_train)
            acc = sum(acc)*1.0/len(y_train)

            acc_test = sum(acc_test)*1.0/len(y_test)
            if epoch%10 == 0 or epoch == args.epoch - 1:
              print('\nEpoch: ', epoch)
              print('\nTraining set: Loss {0:.4f}. Acc {1:.4f}.\nTest set: Acc {2:.4f}.'.format(total_loss, acc, acc_test))

            scheduler.step()

            if acc_test > last_acc:
                torch.save(model.cpu(), model_path_lstm)
                last_acc = acc_test

torch.Size([7000, 132])
torch.Size([7000])
torch.Size([3000, 132])
torch.Size([3000])
218/219
Epoch:  0

Training set: Loss 0.0214. Acc 0.5570.
Test set: Acc 0.5137.
218/219
Epoch:  10

Training set: Loss 0.0135. Acc 0.7831.
Test set: Acc 0.7853.
218/219
Epoch:  20

Training set: Loss 0.0122. Acc 0.8139.
Test set: Acc 0.8040.
218/219
Epoch:  30

Training set: Loss 0.0115. Acc 0.8306.
Test set: Acc 0.8170.
218/219
Epoch:  39

Training set: Loss 0.0110. Acc 0.8376.
Test set: Acc 0.8277.


In [ ]:
#LSTM evaluation
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

model = torch.load(model_path_lstm)
model.to(device)
acc_test = []
y_pred = []

for i in range(num_batch_test):
    st = i * args.batch_size
    ed = min((i+1) * args.batch_size, len(y_test))

    x_test = X_test[st:ed]
    x_test = x_test.transpose(0, 1)
    x_test = torch.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
    x_test = x_test.to(device)

    ftr_test = FTR_test[st:ed]
    ftr_test = ftr_test.transpose(0, 1)
    ftr_test = torch.reshape(ftr_test, (ftr_test.shape[0], ftr_test.shape[1], 1))
    ftr_test = ftr_test.to(device)

    label_test = y_test[st:ed]
    label_test = label_test.to(device)


    output_test = model(x_test, ftr_test)
    predict_test = torch.squeeze(output_test)
    y_pred += predict_test.gt(0.5)

y_pred = list(map(float, y_pred))
y_pred = np.asarray(y_pred)
y_pred = torch.FloatTensor(y_pred)

print('LSTM Results:')
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(metrics.roc_auc_score(y_test, y_pred))

LSTM Results:
Accuracy: 0.8276666666666667
[[1164  358]
 [ 159 1319]]
              precision    recall  f1-score   support

         0.0       0.88      0.76      0.82      1522
         1.0       0.79      0.89      0.84      1478

    accuracy                           0.83      3000
   macro avg       0.83      0.83      0.83      3000
weighted avg       0.83      0.83      0.83      3000

0.8286026860889187
